In [1]:
#Dice
#COMPAS for bail decision
#Adult-Income for income prediction
#German-Credit for assessing credit risk
#Dataset from Lending Club for loan decisions: https://www.lendingclub.com/info/download-data.action

#Alibi

#AIX360

## Importing libraries

In [2]:
import time
import pickle
import dice_ml
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf

from sklearn.impute import *
from sklearn.ensemble import *
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler, LabelEncoder

from imblearn.over_sampling import SMOTENC

from xmoai.problems.objectives import *
from xmoai.problems.restrictions import *
from xmoai.setup.configure import generate_counterfactuals_classification_proba

from alibi.explainers import *

pd.set_option('display.max_columns', None)

In [3]:
tf.compat.v1.disable_eager_execution()
tf.keras.backend.clear_session()
tf.compat.v1.keras.backend.get_session().list_devices()

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, -2072985524047578477),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 9396617216, 4732578360241773610)]

# COMPAS Dataset

In [57]:
df_compas = pd.read_csv('../FairMood/datasets/compas-analysis-master/compas-scores-two-years_filtered.csv')

In [62]:
df_compas

,age,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid
count,6172.000000,6172.000000,6172.000000,6172.000000,6172.000000,6172.000000
mean,34.534511,3.246436,-1.740279,4.418503,0.484446,0.455120
std,11.730938,4.743770,5.084709,2.839463,0.499799,0.498022
min,18.000000,0.000000,-30.000000,1.000000,0.000000,0.000000
25%,25.000000,0.000000,-1.000000,2.000000,0.000000,0.000000
50%,31.000000,1.000000,-1.000000,4.000000,0.000000,0.000000
75%,42.000000,4.000000,-1.000000,7.000000,1.000000,1.000000
max,96.000000,38.000000,30.000000,10.000000,1.000000,1.000000


## Defining required columns

In [5]:
num_indexes = []
cat_indexes = []

num_columns = ['age', 'priors_count', 'days_b_screening_arrest', 'decile_score']
cat_columns = ['c_charge_degree', 'age_cat', 'score_text', 'sex', 'race', 'is_recid']
integer_columns = num_columns
target = 'two_year_recid'

for i in range(df_compas.shape[1]):
    col = df_compas.columns[i]
    if col in num_columns:
        num_indexes.append(i)
    elif col in cat_columns:
        cat_indexes.append(i)

### Converting string-encoded categories to integers

In [6]:
label_encoders = {}

for col in cat_columns:
    encoder = LabelEncoder().fit(df_compas[col])
    label_encoders[col] = encoder
    df_compas[col] = encoder.transform(df_compas[col])

## Training

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df_compas.drop(target, axis=1), df_compas[target], test_size=0.7, random_state=0)
X_train, y_train = SMOTENC(categorical_features=np.isin(df_compas.columns, cat_columns)).fit_resample(X_train, y_train)

In [8]:
one_hot_encode = False

# defining both numeric and categorical transformers
numeric_transformer = Pipeline(
    steps=[("imputer", KNNImputer()), ("scaler", MaxAbsScaler())]
)

# setting-up the preprocessing steps
if one_hot_encode:
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse=False)
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_indexes),
            ("cat", categorical_transformer, cat_indexes),
        ]
    )
else:
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_indexes + cat_indexes),
        ]
    )
    
# defining the model pipeline and training
model = Pipeline(
    steps=[("preprocessor", preprocessor),
           ("classifier", RandomForestClassifier(n_jobs=-1))]
).fit(X_train, y_train)

## Preparing counterfactual generation with DiCE

In [9]:
continuous_precision = {}
for col in num_columns:
    continuous_precision[col] = 0

d = dice_ml.Data(dataframe=pd.concat([X_train, y_train], axis=1),
                 continuous_features=num_columns,
                 continuous_features_precision=continuous_precision,
                 outcome_name=target)
m = dice_ml.Model(model=model, backend='sklearn')
exp = dice_ml.Dice(d,m)

## Preparing counterfactual generation with XMOAI

In [10]:
columns = X_train.columns
categorical_columns_one_hot_encoded = []
categorical_columns_label_encoded = {}

if one_hot_encode and type(model)==Pipeline:
    for cat_col in cat_columns:
        columns_in_cat = [col for col in columns if col.startswith(f'{cat_col}_')]
        columns_in_cat = np.argwhere(np.isin(columns, columns_in_cat)).flatten()
        
        if len(columns_in_cat) > 0:
            categorical_columns_one_hot_encoded.append(columns_in_cat)
else:
    for i in range(len(X_train.columns)):
        if X_train.columns[i] in cat_columns:
            categorical_columns_label_encoded[i] = np.sort(X_train[X_train.columns[i]].unique())
        
display(categorical_columns_one_hot_encoded)
display(categorical_columns_label_encoded)

[]

{1: array([0, 1]),
 2: array([0, 1, 2, 3, 4, 5]),
 3: array([0, 1, 2]),
 4: array([0, 1, 2]),
 5: array([0, 1]),
 9: array([0, 1], dtype=int64)}

In [11]:
# generating counterfactuals
immutable_column_indexes = [] # let's say we can't change the last column
y_acceptable_range = [0.51, 1.0] # we will only accept counterfactuals with the predicted prob. in this range

upper_bounds = np.array(X_train.max(axis=0)*1.0) # this is the maximum allowed number per column
lower_bounds = np.array(X_train.min(axis=0)*1.0) # this is the minimum allowed number per column.
# you may change the bounds depending on the needs specific to the individual being trained.

## Generating counterfactuals

In [12]:
cat_vars_ord = {}
for i in categorical_columns_label_encoded.keys():
    cat_vars_ord[i] = len(np.unique(X_train.values[:, i]))
cat_vars_ord

cf = CounterfactualProto(sess=tf.compat.v1.keras.backend.get_session(),
                         predict=lambda x: model.predict_proba(x),
                         shape=(1,) + X_test.shape[1:],
                         cat_vars=cat_vars_ord,
                         feature_range=(np.array([X_train.min().values]).astype(np.float32), np.array([X_train.max().values]).astype(np.float32)),
                         max_iterations=200,
                         c_steps=3
                        )
cf.fit(X_train.values)

CounterfactualProto(meta={
  'name': 'CounterfactualProto',
  'type': ['blackbox', 'tensorflow', 'keras'],
  'explanations': ['local'],
  'params': {
              'kappa': 0.0,
              'beta': 0.1,
              'feature_range': (array([[ 19.,   0.,   0.,   0.,   0.,   0.,   0., -30.,   1.,   0.]],
      dtype=float32), array([[77.,  1.,  5.,  2.,  2.,  1., 38., 30., 10.,  1.]], dtype=float32)),
              'gamma': 0.0,
              'theta': 0.0,
              'cat_vars': {1: 2, 2: 6, 3: 3, 4: 3, 5: 2, 9: 2},
              'ohe': False,
              'use_kdtree': False,
              'learning_rate_init': 0.01,
              'max_iterations': 200,
              'c_init': 10.0,
              'c_steps': 3,
              'eps': (0.001, 0.001),
              'clip': (-1000.0, 1000.0),
              'update_num_grad': 1,
              'write_dir': None,
              'shape': (1, 10),
              'is_model': False,
              'is_ae': False,
              'is_enc': False,
 

In [13]:
number_of_instances = 150
dice_all_exp = exp.generate_counterfactuals(X_test.iloc[:number_of_instances], total_CFs=100, desired_class="opposite")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [1:01:27<00:00, 24.58s/it]


In [14]:
results = {}

In [55]:
tn, fp, fn, tp = confusion_matrix(y_test.iloc[:number_of_instances], model.predict(X_test.iloc[:number_of_instances])).ravel()
display((tn, fp, fn, tp))

display(y_test[(y_test==1) & (y_test==model.predict(X_test))].index[:5])
display(y_test[(y_test==0) & (y_test==model.predict(X_test))].index[:5])

(79, 5, 1, 65)

Int64Index([1705, 559, 3448, 124, 2678], dtype='int64')

Int64Index([1262, 3228, 4299, 3965, 1669], dtype='int64')

In [25]:
# individual to be evaluated
def calculate_metrics(df, X_current, y_desired, method, categorical_columns_label_encoded,
                      categorical_columns_one_hot_encoded):
    f1, prediction = get_difference_target_classification_proba(model, df, y_desired, method)
    f2 = get_difference_attributes(df.values, X_current.values, ranges,
                                   categorical_columns_label_encoded,
                                   categorical_columns_one_hot_encoded)
    f3 = get_modified_attributes(df, X_current, categorical_columns_one_hot_encoded)
    
    return f1, f2, f3, prediction

ranges = (X_train.max() - X_train.min()).values

for index_to_refer in range(number_of_instances):
    print(f'Processing {index_to_refer}')
    X_current = X_test.iloc[index_to_refer:index_to_refer+1]
    y_desired = int(not bool(np.argmax(model.predict_proba(X_current))))

    # DiCE CFs
    print('Processing DiCE')
    try:
        df_dice = dice_all_exp.cf_examples_list[index_to_refer].final_cfs_df.copy().drop(target, axis=1)

        f1, f2, f3, prediction = calculate_metrics(df_dice, X_current, y_desired, 'predict_proba',
                                                   categorical_columns_label_encoded,
                                                   categorical_columns_one_hot_encoded)

        for col in label_encoders.keys():
            df_dice[col] = label_encoders[col].inverse_transform(df_dice[col].astype(int))

        df_dice = pd.concat([df_dice, pd.DataFrame(np.vstack([f1, f2, f3]).T, columns=['F1', 'F2', 'F3'])], axis=1)
        df_dice['Algorithm'] = 'DiCE'
    except:
        df_dice = pd.DataFrame()
        display('Error in DiCE. Resuming...')
    
    df_dice = pd.DataFrame()
    # XMOAI CFs
    print('Processing XMOAI')
    try:
        front, X_generated, algorithms = generate_counterfactuals_classification_proba(model, X_train,
                                  X_current.iloc[0], y_desired, immutable_column_indexes,
                                  y_acceptable_range, upper_bounds, lower_bounds,
                                  categorical_columns_label_encoded, categorical_columns_one_hot_encoded,
                                  num_indexes, n_gen=100, pop_size=100, max_changed_vars=5,
                                  verbose=True, select_best=True, seed=0)
        
        df_xmoai = pd.DataFrame(X_generated.copy(), columns=X_test.columns)
        for col in label_encoders.keys():
            df_xmoai[col] = label_encoders[col].inverse_transform(df_xmoai[col].astype(int))

        df_xmoai = pd.concat([df_xmoai, pd.DataFrame(front, columns=['F1', 'F2', 'F3'])], axis=1)
        df_xmoai['Algorithm'] = 'Proposal'
    except:
        df_xmoai = pd.DataFrame()
        display('Error in XMOAI. Resuming...')
    
    # Alibi CFs
    print('Processing Alibi')
    try:
        alibi_cf = cf.explain(X_current.values, verbose=False)

        if len(alibi_cf['all'])==0:
            df_alibi = pd.DataFrame()
        else:
            df_alibi = [tuple(v) for v in alibi_cf['all'].values() if len(v) > 0]
            df_alibi = pd.DataFrame([v[0] for v in np.vstack(df_alibi)])
            df_alibi.columns = X_test.columns

            if len(df_alibi)==0:
                continue

            f1, f2, f3, prediction = calculate_metrics(df_alibi, X_current, y_desired, 'predict_proba',
                                                       categorical_columns_label_encoded,
                                                       categorical_columns_one_hot_encoded)

            df_alibi = pd.concat([df_alibi, pd.DataFrame(np.vstack([f1, f2, f3]).T, columns=['F1', 'F2', 'F3'])], axis=1)
            df_alibi['Algorithm'] = 'Alibi'
    except:
        df_alibi = pd.DataFrame()
        display('Error in Alibi. Resuming...')
    
    results[X_current.index[0]] = pd.concat([df_dice, df_alibi, df_xmoai]).reset_index(drop=True)

Processing 0
Processing XMOAI
Generating counterfactuals using UNSGA-III.
n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  |     eps      |  indicator  
    1 |     104 |  0.00000E+00 |  0.285961538 |       2 |            - |            -
    2 |     177 |  0.00000E+00 |  0.276045198 |       2 |  0.00000E+00 |            f
    3 |     178 |  0.00000E+00 |  0.276966292 |       2 |  0.00000E+00 |            f
    4 |     178 |  0.00000E+00 |  0.276966292 |       2 |  0.00000E+00 |            f
Generating counterfactuals using UNSGA-III.
n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  |     eps      |  indicator  
    1 |     252 |  0.00000E+00 |  0.296805556 |       2 |            - |            -
    2 |     552 |  0.00000E+00 |  0.172266667 |       3 |  1.056250000 |        ideal
    3 |     852 |  0.00000E+00 |  0.082766667 |       3 |  0.062500372 |            f
    4 |    1152 |  0.00000E+00 |  0.037400000 |       3 |  0.00000E+00 |            f
    5 |    1452 |  0.0

array([], shape=(0, 3), dtype=float64)

Processing 1
Processing XMOAI
Generating counterfactuals using UNSGA-III.
n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  |     eps      |  indicator  
    1 |     107 |  0.00000E+00 |  0.344579439 |       1 |            - |            -
    2 |     173 |  0.00000E+00 |  0.348670520 |       1 |  0.00000E+00 |            f
    3 |     177 |  0.00000E+00 |  0.350112994 |       1 |  0.00000E+00 |            f
    4 |     178 |  0.00000E+00 |  0.350449438 |       1 |  0.00000E+00 |            f
    5 |     178 |  0.00000E+00 |  0.350449438 |       1 |  0.00000E+00 |            f
Generating counterfactuals using UNSGA-III.
n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  |     eps      |  indicator  
    1 |     269 |  0.00000E+00 |  0.280929368 |       2 |            - |            -
    2 |     569 |  0.00000E+00 |  0.204100000 |       3 |  1.000000000 |        ideal
    3 |     869 |  0.00000E+00 |  0.152666667 |       3 |  0.00000E+00 |            f
    4 |    1169 |  0.0

KeyboardInterrupt: 

In [17]:
import pickle
pickle.dump([results, X_test.iloc[:number_of_instances]], open('compas_results.pkl', 'wb'))